### Mortgage

In [2]:
from absbox import API

localAPI = API("https://absbox.org/api/dev",lang='english',check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.26.6, server:0.26.5

#### No Stress

In [3]:
m = ["Mortgage"
        ,{"originBalance": 12000.0
            ,"originRate": ["fix",0.045]
            ,"originTerm": 30
            ,"freq": "Monthly"
            ,"type": "Level"
            ,"originDate": "2021-02-01"}
        ,{"currentBalance": 10000.0
            ,"currentRate": 0.075
            ,"remainTerm": 20
            ,"status": "Current"}]

In [8]:
r = localAPI.runAsset("2020-01-02"
                     ,[m]
                     ,poolAssump=("Pool"
                                    ,("Mortgage", None, None, None, None)
                                    ,None
                                    ,None)
                     ,read=True)

list(r[0].to_records())

[('2021-12-01', 10000., 0., 0., 0, 0, 0, 0, 0.075, None, None, 0., 0, 0, 0, 0, 0),
 ('2022-01-01', 9529.05, 470.95, 62.5, 0, 0, 0, 0, 0.075, None, None, 470.95, 0, 0, 0, 0, 0),
 ('2022-02-01', 9055.14, 473.91, 59.55, 0, 0, 0, 0, 0.075, None, None, 944.86, 0, 0, 0, 0, 0),
 ('2022-03-01', 8578.28, 476.86, 56.59, 0, 0, 0, 0, 0.075, None, None, 1421.72, 0, 0, 0, 0, 0),
 ('2022-04-01', 8098.43, 479.85, 53.61, 0, 0, 0, 0, 0.075, None, None, 1901.57, 0, 0, 0, 0, 0),
 ('2022-05-01', 7615.59, 482.84, 50.61, 0, 0, 0, 0, 0.075, None, None, 2384.41, 0, 0, 0, 0, 0),
 ('2022-06-01', 7129.72, 485.87, 47.59, 0, 0, 0, 0, 0.075, None, None, 2870.28, 0, 0, 0, 0, 0),
 ('2022-07-01', 6640.83, 488.89, 44.56, 0, 0, 0, 0, 0.075, None, None, 3359.17, 0, 0, 0, 0, 0),
 ('2022-08-01', 6148.87, 491.96, 41.5, 0, 0, 0, 0, 0.075, None, None, 3851.13, 0, 0, 0, 0, 0),
 ('2022-09-01', 5653.84, 495.03, 38.43, 0, 0, 0, 0, 0.075, None, None, 4346.16, 0, 0, 0, 0, 0),
 ('2022-10-01', 5155.71, 498.13, 35.33, 0, 0, 0, 0, 0.075